In [1]:
import math
import tensorflow as tf
import numpy as np
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import os
from tensorflow.contrib import rnn


tf.reset_default_graph()
tf.set_random_seed(777)

if "DISPLAY" not in os.environ:
    mpl.use('TkAgg')

import matplotlib.pyplot as plt

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)

xy = np.genfromtxt('/Users/yeseo/Desktop/City_Counted_TaxiMach_Link_Dataset_Full_201501 - 12.txt',delimiter = ',',dtype = None)
xy_with_noise = np.genfromtxt('/Users/yeseo/Desktop/2015eliminated_1.txt',delimiter = ',',dtype = np.float32)

xy= xy[:,:27]
a = xy[:,:2]
b = xy[:,2:]
b= MinMaxScaler(b)
xy = np.hstack((a,b))

xy_with_noise = xy_with_noise[:,:27]
a_with_noise = xy_with_noise[:,:2]
b_with_noise = xy_with_noise[:,2:]
b_with_noise = MinMaxScaler(b_with_noise)
xy_with_noise = np.hstack((a_with_noise,b_with_noise))



seq_length =6
data_dim =27
hidden_dim = 27
output_dim = 25
learning_rate = 0.01
iterations = 1000
epsilon = 1
e = math.exp(epsilon)
q = (1/(e+1))

train_size = int(len(xy)*0.7)


train_set = xy[:train_size]
test_set = xy[train_size:]

train_set_with_noise = xy_with_noise[:train_size]
test_set_with_noise = xy_with_noise[train_size:]

def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i+seq_length,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

#train_set, test_set 만들기
trainX, trainY = build_dataset(train_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)

trainX_with_noise, trainY_with_noise = build_dataset(train_set_with_noise,seq_length)
testX_with_noise,testY_with_noise = build_dataset(test_set_with_noise, seq_length)


X1 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y1 = tf.placeholder(tf.float32,[None,25])

X2 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y2 = tf.placeholder(tf.float32,[None,25])

#LSTM CELL만들기
with tf.variable_scope("rnn1"):
    cell1 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    multi_cell1 = tf.contrib.rnn.MultiRNNCell([cell1]*2)
    outputs1,_states1 = tf.nn.dynamic_rnn(multi_cell1,X1,dtype = tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs1[:,-1], output_dim,activation_fn = None)

    loss1 =tf.reduce_mean(tf.square(Y_pred-Y1))
    train1 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss1)


with tf.variable_scope("rnn2"):
    cell2 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    multi_cell2 =  tf.contrib.rnn.MultiRNNCell([cell2]*2)
    outputs2,_states2 = tf.nn.dynamic_rnn(multi_cell2, X2, dtype = tf.float32)
    Y_pred_with_noise = tf.contrib.layers.fully_connected(outputs2[:,-1], output_dim,activation_fn = None)

    loss2 =tf.reduce_mean(tf.square(Y_pred_with_noise-Y2))
    train2 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss2)




#RMSE 측정
targets = tf.placeholder(tf.float32,[None,25])
predictions = tf.placeholder(tf.float32,[None,25])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))


x1 = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])*2
x2 = x1+0.33*2
x3 = x2+0.33*2

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iterations):
        _, step_loss1 = sess.run([train1,loss1],feed_dict={X1:trainX, Y1:trainY})
        print("[step: {}] loss: {}".format(i,step_loss1))
        _, step_loss2 = sess.run([train2,loss2],feed_dict={X2:trainX_with_noise, Y2:trainY_with_noise})
        print("[nstep: {}] loss: {}".format(i,step_loss2))
        
    test_predict = sess.run(Y_pred, feed_dict = {X1:testX})
    test_predict_with_noise = sess.run(Y_pred_with_noise, feed_dict = {X2:testX_with_noise})

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
[step: 0] loss: 0.20791023969650269
[nstep: 0] loss: 0.2396225482225418
[step: 1] loss: 0.08533928543329239
[nstep: 1] loss: 0.14774669706821442
[step: 2] loss: 0.04344991594552994
[nstep: 2] loss: 0.1016237810254097
[step: 3] loss: 0.03496082127094269
[nstep: 3] loss: 0.0704653188586235
[step: 4] loss: 0.02889198623597622
[nstep: 4] loss: 0.049962226301431656
[step: 5] loss: 0.025131380185484886
[nstep: 5] loss: 0.03616470843553543
[step: 6] loss: 0.02470528520643711
[nstep: 6] loss: 0.02802380733191967
[step: 7] loss: 0.023841656744480133
[nstep: 7] loss: 0.02445714734494686
[step: 8] loss: 0.02278904616832733
[nstep: 8] loss: 0.023938314989209175
[step: 9] loss: 0.022407419979572296
[nstep: 9] loss: 0.02528502605855465
[step: 10] loss: 0.02164019085466861

[step: 102] loss: 0.0025484671350568533
[nstep: 102] loss: 0.0077240075916051865
[step: 103] loss: 0.0025343764573335648
[nstep: 103] loss: 0.007706062868237495
[step: 104] loss: 0.002432374283671379
[nstep: 104] loss: 0.007722881622612476
[step: 105] loss: 0.0023611823562532663
[nstep: 105] loss: 0.007659499533474445
[step: 106] loss: 0.0023746087681502104
[nstep: 106] loss: 0.007676725275814533
[step: 107] loss: 0.0024224568624049425
[nstep: 107] loss: 0.0076533351093530655
[step: 108] loss: 0.0024379410315304995
[nstep: 108] loss: 0.0076171960681676865
[step: 109] loss: 0.0023866405244916677
[nstep: 109] loss: 0.007637395989149809
[step: 110] loss: 0.00232197274453938
[nstep: 110] loss: 0.007605643477290869
[step: 111] loss: 0.002291165292263031
[nstep: 111] loss: 0.007578135468065739
[step: 112] loss: 0.002303970977663994
[nstep: 112] loss: 0.007594050373882055
[step: 113] loss: 0.0023334852885454893
[nstep: 113] loss: 0.00756797706708312
[step: 114] loss: 0.0023433775641024113
[ns

[step: 206] loss: 0.0019146472914144397
[nstep: 206] loss: 0.007170995231717825
[step: 207] loss: 0.001879936084151268
[nstep: 207] loss: 0.007179704029113054
[step: 208] loss: 0.001847988460212946
[nstep: 208] loss: 0.007180118001997471
[step: 209] loss: 0.001823566621169448
[nstep: 209] loss: 0.0071730040945112705
[step: 210] loss: 0.001812262344174087
[nstep: 210] loss: 0.007159905042499304
[step: 211] loss: 0.001813019160181284
[nstep: 211] loss: 0.007147574797272682
[step: 212] loss: 0.0018219668418169022
[nstep: 212] loss: 0.007139165885746479
[step: 213] loss: 0.0018355248030275106
[nstep: 213] loss: 0.007136365864425898
[step: 214] loss: 0.0018496853299438953
[nstep: 214] loss: 0.007134030107408762
[step: 215] loss: 0.0018653167644515634
[nstep: 215] loss: 0.007130867335945368
[step: 216] loss: 0.0018763156840577722
[nstep: 216] loss: 0.007122465409338474
[step: 217] loss: 0.0018866374157369137
[nstep: 217] loss: 0.007113934960216284
[step: 218] loss: 0.001885475474409759
[nste

[nstep: 309] loss: 0.006980824749916792
[step: 310] loss: 0.001627815654501319
[nstep: 310] loss: 0.007014139089733362
[step: 311] loss: 0.0016360152512788773
[nstep: 311] loss: 0.007010189816355705
[step: 312] loss: 0.0016512639122083783
[nstep: 312] loss: 0.006976197008043528
[step: 313] loss: 0.0016760913422331214
[nstep: 313] loss: 0.006996003910899162
[step: 314] loss: 0.0017238034633919597
[nstep: 314] loss: 0.006997117772698402
[step: 315] loss: 0.0017961455741897225
[nstep: 315] loss: 0.006971574388444424
[step: 316] loss: 0.0019275175873190165
[nstep: 316] loss: 0.006979130674153566
[step: 317] loss: 0.0020652771927416325
[nstep: 317] loss: 0.006987287662923336
[step: 318] loss: 0.0022324619349092245
[nstep: 318] loss: 0.0069688791409134865
[step: 319] loss: 0.0021575940772891045
[nstep: 319] loss: 0.0069657377898693085
[step: 320] loss: 0.0019325498724356294
[nstep: 320] loss: 0.0069768899120390415
[step: 321] loss: 0.0016503656515851617
[nstep: 321] loss: 0.00696745514869689

[nstep: 413] loss: 0.006944021675735712
[step: 414] loss: 0.001505897962488234
[nstep: 414] loss: 0.006918531842529774
[step: 415] loss: 0.001565877697430551
[nstep: 415] loss: 0.006882225163280964
[step: 416] loss: 0.0015545657370239496
[nstep: 416] loss: 0.0068547590635716915
[step: 417] loss: 0.001498222816735506
[nstep: 417] loss: 0.006842921953648329
[step: 418] loss: 0.0015681893564760685
[nstep: 418] loss: 0.006848203483968973
[step: 419] loss: 0.0014987995382398367
[nstep: 419] loss: 0.0068642329424619675
[step: 420] loss: 0.001518694101832807
[nstep: 420] loss: 0.00688175018876791
[step: 421] loss: 0.0015302777756005526
[nstep: 421] loss: 0.006895259488373995
[step: 422] loss: 0.001481629558838904
[nstep: 422] loss: 0.006892537232488394
[step: 423] loss: 0.0015236218459904194
[nstep: 423] loss: 0.006883166264742613
[step: 424] loss: 0.0014915767824277282
[nstep: 424] loss: 0.006865212693810463
[step: 425] loss: 0.0014880847884342074
[nstep: 425] loss: 0.006850242614746094
[ste

[step: 518] loss: 0.001348836231045425
[nstep: 518] loss: 0.006762426812201738
[step: 519] loss: 0.0013479138724505901
[nstep: 519] loss: 0.006759814918041229
[step: 520] loss: 0.001347349607385695
[nstep: 520] loss: 0.006767703220248222
[step: 521] loss: 0.001347556710243225
[nstep: 521] loss: 0.00676352484151721
[step: 522] loss: 0.0013495873427018523
[nstep: 522] loss: 0.006770421285182238
[step: 523] loss: 0.001355750602670014
[nstep: 523] loss: 0.006767727434635162
[step: 524] loss: 0.0013725993921980262
[nstep: 524] loss: 0.006776754278689623
[step: 525] loss: 0.0014137333491817117
[nstep: 525] loss: 0.006770569831132889
[step: 526] loss: 0.0015232007717713714
[nstep: 526] loss: 0.006772890221327543
[step: 527] loss: 0.0017635870026424527
[nstep: 527] loss: 0.0067741782404482365
[step: 528] loss: 0.0023653011303395033
[nstep: 528] loss: 0.0067899515852332115
[step: 529] loss: 0.0030251832213252783
[nstep: 529] loss: 0.006794719956815243
[step: 530] loss: 0.003681695321574807
[nst

[nstep: 622] loss: 0.006689310073852539
[step: 623] loss: 0.0012539628660306334
[nstep: 623] loss: 0.006694287993013859
[step: 624] loss: 0.0012527485378086567
[nstep: 624] loss: 0.006696015130728483
[step: 625] loss: 0.0012514857808128
[nstep: 625] loss: 0.006700778380036354
[step: 626] loss: 0.0012502443278208375
[nstep: 626] loss: 0.006702221930027008
[step: 627] loss: 0.0012489737709984183
[nstep: 627] loss: 0.006707433145493269
[step: 628] loss: 0.0012477646814659238
[nstep: 628] loss: 0.006711528170853853
[step: 629] loss: 0.001246526837348938
[nstep: 629] loss: 0.0067200125195086
[step: 630] loss: 0.001245295163244009
[nstep: 630] loss: 0.006722533609718084
[step: 631] loss: 0.001244048005901277
[nstep: 631] loss: 0.006725884508341551
[step: 632] loss: 0.0012427822221070528
[nstep: 632] loss: 0.00672439020127058
[step: 633] loss: 0.001241543097421527
[nstep: 633] loss: 0.006729337386786938
[step: 634] loss: 0.0012402903521433473
[nstep: 634] loss: 0.006719102617353201
[step: 635

[nstep: 727] loss: 0.006627862341701984
[step: 728] loss: 0.001179158454760909
[nstep: 728] loss: 0.006622412707656622
[step: 729] loss: 0.0011761849746108055
[nstep: 729] loss: 0.006632938515394926
[step: 730] loss: 0.0011761768255382776
[nstep: 730] loss: 0.006639110390096903
[step: 731] loss: 0.0011728104436770082
[nstep: 731] loss: 0.006625941954553127
[step: 732] loss: 0.0011726308148354292
[nstep: 732] loss: 0.006621720269322395
[step: 733] loss: 0.0011701621115207672
[nstep: 733] loss: 0.006621664855629206
[step: 734] loss: 0.0011688306694850326
[nstep: 734] loss: 0.0066126431338489056
[step: 735] loss: 0.0011675192508846521
[nstep: 735] loss: 0.006618532817810774
[step: 736] loss: 0.0011654157424345613
[nstep: 736] loss: 0.0066248346120119095
[step: 737] loss: 0.0011644981568679214
[nstep: 737] loss: 0.006621195934712887
[step: 738] loss: 0.0011625596089288592
[nstep: 738] loss: 0.006621106993407011
[step: 739] loss: 0.0011611244408413768
[nstep: 739] loss: 0.006624273955821991

[nstep: 832] loss: 0.006554704159498215
[step: 833] loss: 0.0010256585665047169
[nstep: 833] loss: 0.0065535674802958965
[step: 834] loss: 0.0010241237469017506
[nstep: 834] loss: 0.006553130690008402
[step: 835] loss: 0.0010225861333310604
[nstep: 835] loss: 0.006553581450134516
[step: 836] loss: 0.0010210431646555662
[nstep: 836] loss: 0.006553176324814558
[step: 837] loss: 0.0010194986825808883
[nstep: 837] loss: 0.006555027328431606
[step: 838] loss: 0.0010179495438933372
[nstep: 838] loss: 0.006555837579071522
[step: 839] loss: 0.0010163964470848441
[nstep: 839] loss: 0.0065598031505942345
[step: 840] loss: 0.0010148398578166962
[nstep: 840] loss: 0.0065660555846989155
[step: 841] loss: 0.001013280125334859
[nstep: 841] loss: 0.006579176522791386
[step: 842] loss: 0.0010117179481312633
[nstep: 842] loss: 0.006597478408366442
[step: 843] loss: 0.001010152860544622
[nstep: 843] loss: 0.006636958569288254
[step: 844] loss: 0.001008585561066866
[nstep: 844] loss: 0.006689417641609907


[nstep: 937] loss: 0.006520143710076809
[step: 938] loss: 0.0008923101704567671
[nstep: 938] loss: 0.006514483597129583
[step: 939] loss: 0.0008909413008950651
[nstep: 939] loss: 0.006510184612125158
[step: 940] loss: 0.0008897992083802819
[nstep: 940] loss: 0.006507136393338442
[step: 941] loss: 0.0008884578128345311
[nstep: 941] loss: 0.0065044863149523735
[step: 942] loss: 0.0008870167075656354
[nstep: 942] loss: 0.006500899326056242
[step: 943] loss: 0.0008857711800374091
[nstep: 943] loss: 0.006497059483081102
[step: 944] loss: 0.0008846154087223113
[nstep: 944] loss: 0.006493529304862022
[step: 945] loss: 0.0008832087041810155
[nstep: 945] loss: 0.0064911190420389175
[step: 946] loss: 0.0008818992646411061
[nstep: 946] loss: 0.006490091793239117
[step: 947] loss: 0.0008807335398159921
[nstep: 947] loss: 0.006490196101367474
[step: 948] loss: 0.0008794774767011404
[nstep: 948] loss: 0.0064908722415566444
[step: 949] loss: 0.0008781580254435539
[nstep: 949] loss: 0.0064916056580841

In [2]:
testY_mean = np.mean(testY)
test_predict_mean = np.mean(test_predict)
test_predict_with_noise_mean = np.mean(test_predict_with_noise)

X_1 = testY-testY_mean
Y_1 = test_predict-test_predict_mean
Y_2 = test_predict_with_noise-test_predict_with_noise_mean
#분자
a_1 = np.sum(np.multiply(X_1,Y_1))
a_2 = np.sum(np.multiply(X_1,Y_2))

#분모
b_1 = np.sqrt(np.sum(np.square(X_1)))
b_2 = b_1
c_1 = np.sqrt(np.sum(np.square(Y_1)))
c_2 = np.sqrt(np.sum(np.square(Y_2)))
#상관계수
r_1 = (a_1/(b_1*c_1))
r_2 = (a_2/(b_2*c_2))
print("correlation(real&predict) :{}".format(r_1))
print("correlation(real&noise) :{}".format(r_2))

correlation(real&predict) :0.9800392012114896
correlation(real&noise) :0.9425082676587109


In [3]:
#Euclidean Distance
ED = abs(testY[:,0]-test_predict[:,0])
print(ED,"\n",np.mean(ED),"\n",sum(ED))

[0.03586297 0.05377722 0.08003187 ... 0.02401246 0.03321214 0.03538657] 
 0.03192697810770819 
 38.44008164168067


In [7]:
rmp = testY[:,0] - test_predict[:,0]
rmp_2 = np.square(rmp)
ED_2 = np.sqrt(sum(rmp_2))
print(ED_2)

1.5451346335281584
